In [1]:
# CELL 0 — Install dependencies (Unsloth-safe + latest transformers)

!pip uninstall -y transformers tokenizers peft -qq
!pip install --quiet --upgrade transformers tokenizers peft unsloth
!pip install --quiet datasets evaluate accelerate bitsandbytes safetensors protobuf==3.20.*

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 99.0 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.2/378.2 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 15.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 1.6 MB/s eta 0:00:00:00:0100:

In [2]:
# CELL 1 — Imports & Global Config

import os, sys, json, math, time
import torch
import pandas as pd
from pathlib import Path
from datetime import datetime
from abc import ABC, abstractmethod

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Clears unused GPU memory cache
torch.cuda.empty_cache()

print("CUDA available:", torch.cuda.is_available())
print("CUDA devices:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(i, torch.cuda.get_device_name(i))

CUDA available: True
CUDA devices: 2
0 Tesla T4
1 Tesla T4


In [3]:
# CELL 2 — HuggingFace Login

from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

hf_token = UserSecretsClient().get_secret("HF_TOKEN")
login(token=hf_token)


In [4]:
# CELL 3 — Load Model & Tokenizer

from unsloth import FastLanguageModel

MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = 4096,      # Maximum number of tokens the model can process in one sequence
    dtype = torch.float16,
    load_in_4bit = True,     # Loads model weights in 4-bit quantization
    token = hf_token,
    device_map = "auto",          # auto-distribute across T4 * 2
    offload_folder = "/kaggle/working/offload",
    use_gradient_checkpointing = True  #  standard PyTorch checkpointing instead of Unsloth patch
)

# Set the padding token to be the same as the end-of-sequence token

tokenizer.pad_token = tokenizer.eos_token


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2026-01-03 04:02:18.024659: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767412938.196472      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767412938.247439      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.10: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [5]:
# CELL 4 — Dataset Processor

from datasets import Dataset
import pandas as pd

class DatasetProcessor:
    def __init__(self, csv_path, tokenizer, max_length=4096, max_train_samples=None):
        self.csv_path = csv_path
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.max_train_samples = max_train_samples

    def load(self):
        df = pd.read_csv(self.csv_path)
        df = df[df["Questions"].notna() & df["Answers"].notna()]
        if self.max_train_samples:
            df = df.sample(min(len(df), self.max_train_samples), random_state=42)
        return Dataset.from_pandas(df)

    def tokenize(self, dataset):
        def tokenize_fn(example):
            prompt = (
                f"বিষয়: {example['Topics']}\n"
                f"শিরোনাম: {example['Question-Title']}\n"
                f"প্রশ্ন: {example['Questions']}\n\n"
                f"সহানুভূতিশীল উত্তর:"
            )
            answer = example["Answers"]
            full_text = prompt + " " + answer

            #  Truncate sequences to model's max_length
            tokenized = self.tokenizer(
                full_text,
                truncation=True,     # Prevents CUDA memory overflow
                padding=False,
                max_length=self.max_length,
            )
            
            # Tokenize prompt separately (without special tokens)
            prompt_ids = self.tokenizer(prompt, add_special_tokens=False)["input_ids"]
            labels = tokenized["input_ids"].copy()
            labels[:len(prompt_ids)] = [-100] * len(prompt_ids)  # Mask prompt tokens

            tokenized["labels"] = labels

            #  Required for evaluation
            tokenized["eval_prompt"] = prompt
            tokenized["eval_answer"] = answer

            return tokenized
            
        # Apply tokenize_fn to entire dataset
        return dataset.map(tokenize_fn, remove_columns=dataset.column_names)

In [6]:
# CELL 5 — Strategy Pattern (Unsloth, LoRA)

#  Abstract base class for fine-tuning strategy
class FineTuneStrategy(ABC):
    @abstractmethod
    def apply(self, model):
        
        """Apply the fine-tuning strategy to the model"""
        pass


# 🔹 LoRA Strategy (Optional PEFT)
class LoRAStrategy(FineTuneStrategy):
    def __init__(self, r=16, alpha=32, dropout=0.05):
        self.r = r
        self.alpha = alpha
        self.dropout = dropout
        self.target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]

    def apply(self, model):
        """
        Apply LoRA-based parameter-efficient fine-tuning.
        """
        model = FastLanguageModel.get_peft_model(
            model,
            r=self.r,
            target_modules=self.target_modules,
            lora_alpha=self.alpha,
            lora_dropout=self.dropout,
            bias="none",
            use_gradient_checkpointing=True,  # standard PyTorch checkpointing
        )
        lora_config = {
            "strategy": "LoRA",
            "r": self.r,
            "alpha": self.alpha,
            "dropout": self.dropout,
            "target_modules": self.target_modules
        }
        return model, lora_config

#  ACTIVE STRATEGY — Unsloth (QUALITY-OPTIMIZED)
class UnslothStrategy(FineTuneStrategy):
    def apply(self, model):
        
        # Attach LoRA adapters to the base LLaMA model
        model = FastLanguageModel.get_peft_model(
            model,
            r = 16,   # Rank controls LoRA capacity 
            target_modules = [
                "q_proj",   # Query projection (attention focus)
                "k_proj",   # Key projection (context matching)
                "v_proj",   # Value projection (information flow)
                "o_proj",   #  Final Output projection
            ],
            lora_alpha = 32,     # Alpha scales LoRA updates
            lora_dropout = 0.05,  # Prevents overfitting
            bias = "none",  # Bias parameters are not trained
            use_gradient_checkpointing = "unsloth",  # memory-efficient
        )
        
        # Required for reproducibility & reporting
        lora_config = {
            "strategy": "Unsloth",
            "r": 16,
            "alpha": 32,
            "dropout": 0.05,
            "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj"],
        }

        return model, lora_config


#  Function to dynamically select strategy
def select_strategy(strategy_name="unsloth"):
    """
    Returns the strategy instance based on the name.
    Options: "unsloth", "lora"
    """
    if strategy_name.lower() == "lora":
        return LoRAStrategy()
    else:
        return UnslothStrategy()


# Dynamically choose strategy
chosen_strategy_name = "unsloth"  # change to "lora" if needed
strategy = select_strategy(chosen_strategy_name)
model, lora_config = strategy.apply(model)

# Print trainable parameters
model.print_trainable_parameters()


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.12.10 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


trainable params: 13,631,488 || all params: 8,043,892,736 || trainable%: 0.1695


In [7]:
# CELL 6 — Data Collator

from torch.nn.utils.rnn import pad_sequence

# This class will be called by HuggingFace Trainer for all training batch

class LLMDataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):

        # Convert input_ids to PyTorch tensors
        input_ids = [torch.tensor(x["input_ids"]) for x in batch]

        # Convert labels to PyTorch tensors
        labels = [torch.tensor(x["labels"]) for x in batch]

        input_ids = pad_sequence(input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        labels = pad_sequence(labels, batch_first=True, padding_value=-100)

        # Create attention mask
        attention_mask = input_ids.ne(self.tokenizer.pad_token_id)

        return {
            "input_ids": input_ids,
            "labels": labels,
            "attention_mask": attention_mask,
        }


In [8]:
# CELL 7 — LLAMAFineTuner

from transformers import Trainer, TrainingArguments

class LLAMAFineTuner:
    def __init__(self, model, tokenizer, train_ds):
        self.model = model        # Store model (already wrapped with Unsloth LoRA)
        self.tokenizer = tokenizer      # Store tokenizer (needed by Trainer for padding & saving)
        self.train_ds = train_ds        # Store training dataset (tokenized HuggingFace Dataset)
        self.checkpoint_dir = "/kaggle/working/checkpoints"

    # This method sets up training configuration and launches training
    def train(self):
        args = TrainingArguments(
                output_dir=self.checkpoint_dir,
                per_device_train_batch_size=1,  # Batch size per GPU
                gradient_accumulation_steps=4,  # Accumulate gradients over 4 steps
                num_train_epochs=2,  # Number of full passes over the dataset
                learning_rate=2e-4,  
                fp16=True,  # Enables half-precision training
                save_strategy="steps",  # Save model checkpoints based on step count
                save_steps=600,  
                logging_steps=500,
                report_to="none",  # Disable logging to external services
                optim="adamw_torch",  # Uses PyTorch-native AdamW
                dataloader_pin_memory=False,  # Prevents CUDA memory fragmentation on Kaggle
            )


        trainer = Trainer(
            model=self.model,
            args=args,
            train_dataset=self.train_ds,
            tokenizer=self.tokenizer,
            data_collator=LLMDataCollator(self.tokenizer),
        )

        return trainer.train()


In [9]:
# CELL 8 — Evaluator

import evaluate
import math
import torch

class Evaluator:
    def __init__(self, tokenizer, model):
        self.bleu = evaluate.load("bleu")
        self.rouge = evaluate.load("rouge")
        self.tokenizer = tokenizer
        self.model = model

    def perplexity(self, loss):
        """Compute perplexity from training loss"""
        return math.exp(loss)

    def text_metrics(self, dataset, samples=50):
        """
        Compute BLEU and ROUGE metrics for generated text.
        Ensures prompt is removed from prediction before comparison.
        """
        self.model.eval()
        preds, refs = [], []

        # Limit to 'samples' number of examples
        for i in range(min(samples, len(dataset))):
            item = dataset[i]

            # Encode the prompt only
            inputs = self.tokenizer(
                item["eval_prompt"],
                return_tensors="pt",
                truncation=True,
                max_length=self.tokenizer.model_max_length
            ).to(self.model.device)

            with torch.no_grad():
                out = self.model.generate(
                    **inputs,
                    max_new_tokens=150,
                    do_sample=False  # deterministic generation for evaluation
                )

            # Decode output
            pred_text = self.tokenizer.decode(out[0], skip_special_tokens=True)

            #  Strip prompt from generated text
            if pred_text.startswith(item["eval_prompt"]):
                pred_text = pred_text[len(item["eval_prompt"]):].strip()

            preds.append(pred_text)
            refs.append([item["eval_answer"].strip()])

        # Compute BLEU (strict n-gram overlap)
        bleu_score = self.bleu.compute(predictions=preds, references=refs)["bleu"]

        # Compute ROUGE
        rouge_scores = self.rouge.compute(predictions=preds, references=refs)

        return {
            "bleu": bleu_score,
            "rouge": rouge_scores
        }


In [10]:
# CELL 9 — Train + Evaluate + Metrics Table

CSV_PATH = "/kaggle/input/bengali-empathetic-conversations-corpus/BengaliEmpatheticConversationsCorpus .csv"

processor = DatasetProcessor(CSV_PATH, tokenizer, max_train_samples=6000)
dataset = processor.tokenize(processor.load())

tuner = LLAMAFineTuner(model, tokenizer, dataset)
train_output = tuner.train()

# 🔹 Evaluation
torch.cuda.empty_cache()
evaluator = Evaluator(tokenizer, model)
text_metrics = evaluator.text_metrics(dataset)

metrics = {
    "perplexity": evaluator.perplexity(train_output.training_loss),
    "bleu": text_metrics["bleu"],
    "rouge-1": text_metrics["rouge"]["rouge1"], # ROUGE-1: unigram overlap
    "rouge-2": text_metrics["rouge"]["rouge2"], # ROUGE-2: bigram overlap
    "rouge-l": text_metrics["rouge"]["rougeL"], # ROUGE-L: longest common subsequence
} 

#  Evaluation table
eval_table = pd.DataFrame(
    metrics.items(),
    columns=["Metric", "Score"]
)

print("\n=== Evaluation Metrics ===")
display(eval_table)


Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

/tmp/ipykernel_47/2196880445.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer._unsloth___init__`. Use `processing_class` instead.
  trainer = Trainer(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 2
   \\   /|    Num examples = 6,000 | Num Epochs = 2 | Total steps = 3,000
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 13,631,488 of 8,043,892,736 (0.17% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
500,0.736000
1000,0.707200
1500,0.685200
2000,0.646000
2500,0.644500
3000,0.645300



=== Evaluation Metrics ===


,Metric,Score
0,perplexity,1.968716
1,bleu,0.008530
2,rouge-1,0.000000
3,rouge-2,0.000000
4,rouge-l,0.000000


In [11]:
# CELL 10 — Save Experiment Logs

experiment_log = {
    "id": 1,
    "model_name": MODEL_NAME,
    "lora_config": lora_config,
    "train_loss": train_output.training_loss,
    "metrics": metrics,
    "timestamp": datetime.now().isoformat()
}

Path("/kaggle/working/LLAMAExperiments.json").write_text(
    json.dumps(experiment_log, indent=2, ensure_ascii=False)
)


491

In [12]:
# CELL 11 — Save Final Model
SAVE_DIR = "/kaggle/working/llama_unsloth_final"
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)


('/kaggle/working/llama_unsloth_final/tokenizer_config.json',
 '/kaggle/working/llama_unsloth_final/special_tokens_map.json',
 '/kaggle/working/llama_unsloth_final/chat_template.jinja',
 '/kaggle/working/llama_unsloth_final/tokenizer.json')

In [13]:
# CELL 12 — Human Evaluation Generation
prompts = [
    "আমি খুব দুঃখিত, কেউ আমাকে বোঝে না।",
    "আমি আজ খুব খুশি, কারণ আমার পরিবার আমাকে সমর্থন করে।"
]

responses = []

for i, p in enumerate(prompts):
    inputs = tokenizer(p, return_tensors="pt").to(model.device)
    out = model.generate(**inputs, max_new_tokens=200)
    text = tokenizer.decode(out[0], skip_special_tokens=True)

    responses.append({
        "id": i + 1,
        "experiment_id": 1,
        "input_text": p,
        "response_text": text,
        "timestamp": datetime.now().isoformat(),
        "human_score": None
    })

Path("/kaggle/working/GeneratedResponses.json").write_text(
    json.dumps(responses, indent=2, ensure_ascii=False)
)



857

In [14]:
# CELL 13 — Sample Model Response on Test Prompt (Printed Output)

test_prompt = "আমি চেষ্টা করছি কিন্তু কিছুতেই সফল হতে পারছি না, আত্মবিশ্বাস হারিয়ে ফেলেছি।"

model.eval()

inputs = tokenizer(
    test_prompt,
    return_tensors="pt"
).to(model.device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )

# Extract only newly generated tokens
generated_ids = output[0][inputs["input_ids"].shape[1]:]
response_text = tokenizer.decode(generated_ids, skip_special_tokens=True)

print("\n================ SAMPLE MODEL RESPONSE ================\n")
print("🟢 Test Prompt:")
print(test_prompt)

print("\n🟢 Model Response:")
print(response_text)
print("\n======================================================\n")



================ SAMPLE MODEL RESPONSE ================

🟢 Test Prompt:
আমি চেষ্টা করছি কিন্তু কিছুতেই সফল হতে পারছি না, আত্মবিশ্বাস হারিয়ে ফেলেছি।

🟢 Model Response:
 আমি আপনার সাথে পুরোপুরি একমত! আমি পুরোপুরি প্রস্তুত নই। কিন্তু আমি নিশ্চিত যে আমি যদি প্রস্তুত হতে পারি, তাহলে এটি এখনও একটি চালাক বা অত্যন্ত বিপজ্জনক অভিজ্ঞতা হ�


